In [15]:
import numpy as np
import pandas as pd

### gene expression data 
gpl = np.load('data/microarray/gpl570_expression.npy') #microarray samples - older and shittier
refine = np.load('data/RNAseq/filtered_refine.bio_tpm_expression.npy') #RNAseq samples - new technology

### age, sex, and other sample metadata for each sample id
gpl_labels = pd.read_csv('data/microarray/sample-filtered_manually_annotated_gpl570_age-sex_sample_labels.tsv', sep='\t', encoding='ISO-8859-1')
refine_labels = pd.read_csv('data/RNAseq/sample-filtered_manually_annotated_refine.bio_age-sex_sample_labels.tsv', sep='\t', encoding='ISO-8859-1')

In [16]:
print(gpl.shape)
print(refine.shape)
print(gpl_labels.shape)
print(refine_labels.shape)

(16107, 18478)
(14232, 25570)
(16107, 8)
(14232, 10)


In [29]:
gpl_labels[:5]

,gse,gsm,sex,age,age_group,fine_age_group,tissue,disease
0,GSE11151,GSM281311,female,NaN,fetus,fetus,D014551:Urinary Tract|D014566:Urogenital System,NaN
1,GSE11151,GSM281312,male,NaN,fetus,fetus,D014551:Urinary Tract|D014566:Urogenital System,NaN
2,GSE12621,GSM315989,female,19wk,fetus,fetus,D005123:Eye|D012679:Sense Organs,NaN
3,GSE12621,GSM315990,male,19wk,fetus,fetus,D005123:Eye|D012679:Sense Organs,NaN
4,GSE12621,GSM315991,female,19wk,fetus,fetus,D005123:Eye|D012679:Sense Organs,NaN


In [30]:
refine_labels[:5]

,sample,run,experiment,sex,age,age_group,fine_age_group,tissue,cells,disease
0,ERS2615349,ERR2704713,ERP109940,female,12wk,fetus,fetus,NaN,NaN,normal
1,ERS2615350,ERR2704714,ERP109940,male,12wk,fetus,fetus,NaN,NaN,normal
2,ERS2615351,ERR2704715,ERP109940,male,16wk,fetus,fetus,NaN,NaN,normal
3,ERS2615352,ERR2704716,ERP109940,male,16wk,fetus,fetus,NaN,NaN,normal
4,ERS2615354,ERR2704718,ERP109940,female,9wk,fetus,fetus,NaN,NaN,normal


In [20]:
# Randomly select 5 columns
gpl_random_cols = np.random.choice(gpl.shape[1], 5, replace=False)
refine_random_cols = np.random.choice(refine.shape[1], 5, replace=False)

# Compute summary statistics
gpl_summary = {f"Column {col}": {
    "mean": np.mean(gpl[:, col]),
    "std": np.std(gpl[:, col]),
    "min": np.min(gpl[:, col]),
    "max": np.max(gpl[:, col])
} for col in gpl_random_cols}

refine_summary = {f"Column {col}": {
    "mean": np.mean(refine[:, col]),
    "std": np.std(refine[:, col]),
    "min": np.min(refine[:, col]),
    "max": np.max(refine[:, col])
} for col in refine_random_cols}

# Print the summaries
print("GPL (microarray) Summary:")
for col, stats in gpl_summary.items():
    print(col, stats)

print("\nRefine (RNAseq) Summary:")
for col, stats in refine_summary.items():
    print(col, stats)


GPL (microarray) Summary:
Column 9109 {'mean': 12.480291823213761, 'std': 0.680430943570284, 'min': 4.9842064528086905, 'max': 14.499726539010402}
Column 12657 {'mean': 6.745498146112469, 'std': 1.1114288813265727, 'min': 3.82853712334982, 'max': 10.523073465635699}
Column 1156 {'mean': 6.045624064282047, 'std': 1.0112465717009151, 'min': 4.13875937013015, 'max': 10.638822530382098}
Column 294 {'mean': 5.261984642452963, 'std': 0.812698874230374, 'min': 3.97796353828031, 'max': 11.0528370500697}
Column 14144 {'mean': 7.4581728209158795, 'std': 0.8794532278112108, 'min': 3.8241973489885, 'max': 12.081155023257}

Refine (RNAseq) Summary:
Column 17921 {'mean': 5.63822037064362, 'std': 15.047018918298683, 'min': 0.0, 'max': 465.491334}
Column 25477 {'mean': 13.041578083825183, 'std': 46.83858584414753, 'min': 0.0, 'max': 363.195231}
Column 12986 {'mean': 32.83946157651771, 'std': 23.208142482865995, 'min': 0.0, 'max': 181.94722099999998}
Column 15423 {'mean': 3.8994348126053957, 'std': 40.

#### ONLY IF COMBINING RNAseq with microarray --> drop all the genes (columns) that are not shared between gpl and refine

In [ ]:
# # Load gene IDs from both files
# gene_ids_common = np.loadtxt('data/microarray/common_Entrez_IDs_gpl570-refine.bio.txt', dtype=str) # this is techincally gpl/microarray. just old name and file mix ups type shit
# gene_ids_refine = np.loadtxt('data/RNAseq/refine.bio_geneIDs.txt', dtype=str)

# # Find the common gene IDs
# common_genes = np.intersect1d(gene_ids_refine, gene_ids_common)

# # Identify the columns corresponding to the common genes in both arrays
# common_columns_gpl = np.isin(gene_ids_common, common_genes)
# common_columns_refine = np.isin(gene_ids_refine, common_genes)

# # Filter the original expression data (gpl and refine)
# gpl = gpl[:, common_columns_gpl]
# refine = refine[:, common_columns_refine]

# print(gpl.shape)
# print(refine.shape)

(16107, 18478)
(14232, 18478)


#### also drop columns that have 0 for mean and std

In [22]:
# Find columns in gpl where both mean and std are 0
zero_mean_std_cols_gpl = np.where((np.mean(gpl, axis=0) == 0) & (np.std(gpl, axis=0) == 0))[0]
print("Columns in gpl with mean and std of 0:", zero_mean_std_cols_gpl)

# Find columns in refine where both mean and std are 0
zero_mean_std_cols_refine = np.where((np.mean(refine, axis=0) == 0) & (np.std(refine, axis=0) == 0))[0]
print("Columns in refine with mean and std of 0:", zero_mean_std_cols_refine)

print(len(zero_mean_std_cols_refine))

Columns in gpl with mean and std of 0: []
Columns in refine with mean and std of 0: [    6    21    23    29    30    45    52    71    77    79   101   131
   152   169   179   216   219   226   231   236   242   248   274   279
   280   296   303   317   319   323   333   345   346   363   372   378
   383   384   386   394   397   398   402   406   408   409   414   418
   419   420   421   429   436   440   459   461   462   463   464   465
   466   467   469   470   472   473   474   475   476   478   506   520
   521   523   529   541   555   568   569   572   579   582   584   590
   600   606   614   615   616   619   624   628   672   757   764   771
   822   830   834   845   848   850   858   862   899   933   946   970
   971   986  1015  1128  1221  1253  1274  1310  1387  1394  1404  1407
  1410  1469  1541  1621  1661  1712  1717  1797  1880  1976  1985  2021
  2032  2067  2086  2349  2391  2401  2402  2403  2427  2456  2476  2533
  2563  2596  2604  2700  2826  2893  29

In [23]:
# Drop em now
zero_mean_std_cols_refine = np.where((np.mean(refine, axis=0) == 0) & (np.std(refine, axis=0) == 0))[0]
# Drop those columns from refine
refine = np.delete(refine, zero_mean_std_cols_refine, axis=1)

print("New shape of refine:", refine.shape)

New shape of refine: (14232, 24802)


#### keep track of the column (geneID) names in RNAseq by saving these updated columns to new txt file

In [24]:
# with open('jawn.txt', 'r') as file:
with open('data/RNAseq/refine.bio_geneIDs.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines()]

# Remove the gene IDs corresponding to the zero-mean/zero-std columns
filtered_gene_ids = [gene_id for i, gene_id in enumerate(lines) if i not in zero_mean_std_cols_refine]

# Save the filtered gene IDs to a new file
with open('data/RNAseq/UPDATED_refine.bio_geneIDs.txt', 'w') as f:
    f.write('\n'.join(filtered_gene_ids))

### Now standardize the values . 
#### gpl (microarray) doesn't have to be standardized because .....
#### refine (RNAseq) needs to have arcsin applied to it because .....

In [25]:
refine_normed = np.arcsinh(refine)

#### compare what original vs normalized refine dataset looks like

In [28]:
# Randomly select 5 columns
random_cols = np.random.choice(refine.shape[1], 5, replace=False)

# Compute summary statistics
refine_summary = {f"Column {col}": {
    "mean": np.mean(refine[:, col]),
    "std": np.std(refine[:, col]),
    "min": np.min(refine[:, col]),
    "max": np.max(refine[:, col])
} for col in random_cols}

refine_normed_summary = {f"Column {col}": {
    "mean": np.mean(refine_normed[:, col]),
    "std": np.std(refine_normed[:, col]),
    "min": np.min(refine_normed[:, col]),
    "max": np.max(refine_normed[:, col])
} for col in random_cols}

# Print the summaries
print("Refine Summary:")
for col, stats in refine_summary.items():
    print(col, stats)

print("\nRefine Normed Summary:")
for col, stats in refine_normed_summary.items():
    print(col, stats)

Refine Summary:
Column 12209 {'mean': 0.13948403499156828, 'std': 0.4279829262725154, 'min': 0.0, 'max': 27.250778999999998}
Column 14141 {'mean': 34.6074455054806, 'std': 234.02659167854918, 'min': 0.0, 'max': 9169.115392999998}
Column 14026 {'mean': 5.59550573826588, 'std': 28.52327499730319, 'min': 0.0, 'max': 778.647848}
Column 7322 {'mean': 0.3206617674957841, 'std': 1.007074318443456, 'min': 0.0, 'max': 65.752484}
Column 461 {'mean': 2.118664170460933, 'std': 8.625259581743737, 'min': 0.0, 'max': 301.384425}

Refine Normed Summary:
Column 12209 {'mean': 0.12214490065543492, 'std': 0.25358664634737155, 'min': 0.0, 'max': 3.998565771290158}
Column 14141 {'mean': 2.193795391889703, 'std': 1.9430971229562706, 'min': 0.0, 'max': 9.816743276628058}
Column 14026 {'mean': 0.9002072176162816, 'std': 1.2994130597675237, 'min': 0.0, 'max': 7.35070648008168}
Column 7322 {'mean': 0.25239339809026917, 'std': 0.3768150922492418, 'min': 0.0, 'max': 4.8791024503620735}
Column 461 {'mean': 0.83645

### Now create the response variable, binned age

In [32]:
# first drop the rows with N/A age
gpl_labels2 = gpl_labels.dropna(subset=['age'])
gpl2 = gpl[gpl_labels2.index]

print(len(gpl_labels))
print(len(gpl))
print(len(gpl_labels2))
print(len(gpl2))

16107
16107
16057
16057


In [33]:
# same with refine
refine_labels2 = refine_labels.dropna(subset=['age'])
refine_normed2 = refine_normed[refine_labels2.index]

print(len(refine_labels))
print(len(refine_normed))
print(len(refine_labels2))
print(len(refine_normed2))

14232
14232
14206
14206


In [34]:
# create the binned age category
gpl_labels2 = gpl_labels2.copy()
gpl_labels2['age_in_years'] = gpl_labels2['age'].apply(
    lambda age: 
    float(age.replace('wk', '').strip()) * 7 / 365 if isinstance(age, str) and 'wk' in age 
    else (float(age) if isinstance(age, (int, float)) or (isinstance(age, str) and age.replace('.', '', 1).isdigit()) #1 argument ensures that only one occurrence of . is removed
    else None)
)

bins = [0, 2, 8, 12, 20, 35, 45, 60, 70, 80, float('inf')]
labels = ['[0–2]', '(2–8]', '(8–12]', '(12–20]', '(20–35]', '(35–45]', '(45–60]', '(60–70]', '(70–80]', '> 80']

gpl_labels2['age_bin'] = pd.cut(gpl_labels2['age_in_years'], bins=bins, labels=labels, right=True)

In [35]:
# same for refine
refine_labels2 = refine_labels2.copy()
refine_labels2['age_in_years'] = refine_labels2['age'].apply(
    lambda age: 
    float(age.replace('wk', '').strip()) * 7 / 365 if isinstance(age, str) and 'wk' in age 
    else (float(age) if isinstance(age, (int, float)) or (isinstance(age, str) and age.replace('.', '', 1).isdigit()) #1 argument ensures that only one occurrence of . is removed
    else None)
)

# Map nonstandard age categories to approximate values in years
age_mapping = {
    'Carnegie Stage 23': 0.16,  # ~ 8 weeks (~0.16 years)
    'Carnegie Stage 22': 0.15,  # ~ 7.5 weeks
    'Carnegie Stage 21': 0.14,  # ~ 7 weeks
    'Carnegie Stage21': 0.14,  # account for the typo
    'Carnegie Stage 20': 0.13,  # ~ 6.5 weeks
    'Carnegie Stage 19': 0.12,  # ~ 6 weeks
    'Carnegie Stage 18': 0.11,  # ~ 5.5 weeks
    'Carnegie Stage 17': 0.10,  # ~ 5 weeks
    'Carnegie Stage 16': 0.09,  # ~ 4.5 weeks
    'Carnegie Stage 15': 0.08,  # ~ 4 weeks
    'Carnegie Stage 14': 0.07,  # ~ 3.5 weeks
    'Carnegie Stage 13': 0.06,  # ~ 3 weeks
    'fetus': 0.17,              # ~ 9 weeks
    '9 post conception weeks': 0.17,
    '10 post conception weeks': 0.19,
    '11 post conception weeks': 0.21,
    '12 post conception weeks': 0.23,
    '13 post conception weeks': 0.25,
    '14 post conception weeks': 0.27,
    '15 post conception weeks': 0.29,
    '16 post conception weeks': 0.31,
    '17 post conception weeks': 0.33,
    '19 post conception weeks': 0.37,
    '20 post conception weeks': 0.39,
    'Late 8 post conception weeks': 0.15,
    '0days': 0.0,               # Newborn
    '1days': 0.0027,            # 1 day old (approx. years)
    '3days': 0.008,             # 3 days old
    '7days': 0.02,              # 7 days old
    '18-24': 21,                # Midpoint of range
    '25-30': 27.5,
    '31-36': 33.5,
    '37-42': 39.5,
    '43-50': 46.5,
    '51-60': 55.5,
    '< 1': 0.5                  # Less than 1 year old
}

# Fill nonstandard ages with mapped values
refine_labels2['age_in_years'] = refine_labels2['age_in_years'].fillna(refine_labels2['age'].replace(age_mapping))

bins = [0, 2, 8, 12, 20, 35, 45, 60, 70, 80, float('inf')]
labels = ['[0–2]', '(2–8]', '(8–12]', '(12–20]', '(20–35]', '(35–45]', '(45–60]', '(60–70]', '(70–80]', '> 80']

refine_labels2['age_bin'] = pd.cut(refine_labels2['age_in_years'], bins=bins, labels=labels, right=True, include_lowest=True)

/var/folders/j7/ddlht6kj6dj0_8kvm5tg6q040000gn/T/ipykernel_16194/1183150525.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  refine_labels2['age_in_years'] = refine_labels2['age_in_years'].fillna(refine_labels2['age'].replace(age_mapping))


In [36]:
# Convert the age_bin column to categorical codes (integer labels)
gpl_labels2['age_bin_label'] = pd.Categorical(gpl_labels2['age_bin']).codes
refine_labels2['age_bin_label'] = pd.Categorical(refine_labels2['age_bin']).codes

In [37]:
gpl_labels2['age_bin_label'].value_counts()

age_bin_label
6    4420
7    2910
5    2058
4    2007
8    1817
3     686
9     627
1     588
0     500
2     444
Name: count, dtype: int64

In [38]:
refine_labels2['age_bin_label'].value_counts()

age_bin_label
6    2711
3    2157
4    1867
7    1554
5    1404
0    1243
2    1098
8     929
1     760
9     483
Name: count, dtype: int64

In [39]:
print(len(refine_labels2))
print(len(refine_normed2))
print()
print(len(gpl_labels2))
print(len(gpl2))

14206
14206

16057
16057


### Also make a dataset that was not arcsin transformed, just to check its performance in models vs the normalized datasets

In [ ]:
refine2 = refine[refine_labels2.index]

: 

In [23]:
print(len(refine2))
print(len(refine_labels2))

14206
14206


### Ordinal Regression

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from coral_pytorch.losses import corn_loss
from coral_pytorch.dataset import corn_label_from_logits

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

In [182]:
# Split into 80% train and 20% test
labels = refine_labels2['age_bin_label'].values

X_train, X_test, y_train, y_test = train_test_split(
    refine2, labels, test_size=0.2, random_state=42, stratify=labels
)

In [183]:
# ## "after applyin arcsin this doesnt matter" - arjun

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
class GeneExpressionDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
class MLP(nn.Module):
    def __init__(self, input_size, hidden_units, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_units[0]),
            nn.ReLU(),
            nn.Linear(hidden_units[0], hidden_units[1]),
            nn.ReLU(),
            nn.Linear(hidden_units[1], num_classes - 1)  # CORN reduces classes by 1
        )
    
    def forward(self, x):
        return self.model(x)

In [186]:
INPUT_SIZE = X_train.shape[1]
HIDDEN_UNITS = [64, 32]
NUM_CLASSES = 10  # 10 age bins

model = MLP(INPUT_SIZE, HIDDEN_UNITS, NUM_CLASSES)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

BATCH_SIZE = 128

train_dataset = GeneExpressionDataset(X_train, y_train)
test_dataset = GeneExpressionDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

NUM_EPOCHS = 20

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_features)
        loss = corn_loss(outputs, batch_labels, num_classes=NUM_CLASSES)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * batch_features.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {epoch_loss:.4f}')

model.eval()
total_mae = 0.0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)
        preds = corn_label_from_logits(outputs)
        mae = torch.mean(torch.abs(preds - batch_labels.float()))
        total_mae += mae.item() * batch_features.size(0)

test_mae = total_mae / len(test_loader.dataset)
print(f'Test MAE: {test_mae:.4f}')

### refine (RNAseq)

#### MAE of arcsin with scaler: 0.7241, 0.6780, 0.7220
#### MAE of arcsin without scaler: 0.7942, 0.9078, 0.7829
#### MAE of raw with scaler: 0.7270, 0.6844, 0.7009
#### MAE of raw without scaler: 1.0271, 0.9799

### gpl (microarray)

#### MAE of raw with scaler 1.4044, 1.3615
#### MAE of raw without scaler 1.4801, 1.4903

## 5-fold CV CORN

In [26]:
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import joblib  # To save the indices and MAEs to a file

INPUT_SIZE = refine2.shape[1]  # Use refine2 for input size
HIDDEN_UNITS = [64, 32]
NUM_CLASSES = 10  # 10 age bins
BATCH_SIZE = 128
NUM_EPOCHS = 20

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

labels = refine_labels2['age_bin_label'].values

# Save indices and MAE for reproducibility
fold_indices = {'train': [], 'test': []}
fold_mae = []

# Perform 5-fold cross-validation
for fold, (train_idx, test_idx) in enumerate(skf.split(refine2, labels)):
    print(f"Fold {fold + 1}")
    
    # Save train and test indices for this fold
    fold_indices['train'].append(train_idx)
    fold_indices['test'].append(test_idx)
    
    # Create train and test sets using numpy array slicing
    X_train, X_test = refine2[train_idx], refine2[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Reinitialize the model for each fold
    model = MLP(INPUT_SIZE, HIDDEN_UNITS, NUM_CLASSES)
    
    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Reinitialize optimizer for each fold
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Define datasets and dataloaders
    train_dataset = GeneExpressionDataset(X_train, y_train)
    test_dataset = GeneExpressionDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Training loop
    for epoch in range(NUM_EPOCHS):
        model.train()
        running_loss = 0.0
        for batch_features, batch_labels in train_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = corn_loss(outputs, batch_labels, num_classes=NUM_CLASSES)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * batch_features.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {epoch_loss:.4f}')

    # Evaluation
    model.eval()
    total_mae = 0.0
    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            outputs = model(batch_features)
            preds = corn_label_from_logits(outputs)
            mae = torch.mean(torch.abs(preds - batch_labels.float()))
            total_mae += mae.item() * batch_features.size(0)

    test_mae = total_mae / len(test_loader.dataset)
    print(f'Test MAE for fold {fold + 1}: {test_mae:.4f}')
    
    # Save the trained model for this fold
    torch.save(model.state_dict(), f'model_fold_{fold + 1}.pth')
    
    # Save the MAE for this fold
    fold_mae.append(test_mae)
    
# Save the indices and MAEs to a file for reproducibility
joblib.dump({'indices': fold_indices, 'mae': fold_mae}, 'fold_data.pkl')

Fold 1
Epoch 1/20, Loss: 0.3182
Epoch 2/20, Loss: 0.2157
Epoch 3/20, Loss: 0.1873
Epoch 4/20, Loss: 0.1683
Epoch 5/20, Loss: 0.1565
Epoch 6/20, Loss: 0.1495
Epoch 7/20, Loss: 0.1380
Epoch 8/20, Loss: 0.1327
Epoch 9/20, Loss: 0.1252
Epoch 10/20, Loss: 0.1209
Epoch 11/20, Loss: 0.1160
Epoch 12/20, Loss: 0.1103
Epoch 13/20, Loss: 0.1071
Epoch 14/20, Loss: 0.0997
Epoch 15/20, Loss: 0.0999
Epoch 16/20, Loss: 0.0923
Epoch 17/20, Loss: 0.0906
Epoch 18/20, Loss: 0.0888
Epoch 19/20, Loss: 0.0844
Epoch 20/20, Loss: 0.0817
Test MAE for fold 1: 0.6882
Fold 2
Epoch 1/20, Loss: 0.3293
Epoch 2/20, Loss: 0.2133
Epoch 3/20, Loss: 0.1832
Epoch 4/20, Loss: 0.1667
Epoch 5/20, Loss: 0.1542
Epoch 6/20, Loss: 0.1453
Epoch 7/20, Loss: 0.1400
Epoch 8/20, Loss: 0.1308
Epoch 9/20, Loss: 0.1239
Epoch 10/20, Loss: 0.1177
Epoch 11/20, Loss: 0.1132
Epoch 12/20, Loss: 0.1125
Epoch 13/20, Loss: 0.1064
Epoch 14/20, Loss: 0.1019
Epoch 15/20, Loss: 0.1005
Epoch 16/20, Loss: 0.0934
Epoch 17/20, Loss: 0.0898
Epoch 18/20, L

['fold_data.pkl']

## Model Performance for each age bin - is it better for certain age bins?

In [27]:
refine_labels2['age_bin'].value_counts()

age_bin
(45–60]    2711
(12–20]    2157
(20–35]    1867
(60–70]    1554
(35–45]    1404
[0–2]      1243
(8–12]     1098
(70–80]     929
(2–8]       760
> 80        483
Name: count, dtype: int64

In [28]:
import torch
import numpy as np
import joblib

# Define the response variable (target variable)
unique_age_bin_labels = np.unique(refine_labels2['age_bin_label'].values)

# Initialize a dictionary to hold the sum of absolute errors and counts for each label
error_sums = {label: 0 for label in unique_age_bin_labels}
counts = {label: 0 for label in unique_age_bin_labels}

# Loop through each saved model (for each fold)
for fold in range(1, 6):  # Assuming 5-fold models are saved
    # Load the saved model for this fold
    model = MLP(INPUT_SIZE, HIDDEN_UNITS, NUM_CLASSES)
    model.load_state_dict(torch.load(f'model_fold_{fold}.pth'))
    model.to(device)
    model.eval()

    # Load the corresponding test set indices
    fold_data = joblib.load('fold_data.pkl')
    test_idx = fold_data['indices']['test'][fold - 1]  # Get the test indices for this fold
    X_test = refine2[test_idx]  # Get the test features
    y_test = labels[test_idx]   # Get the test labels (age_bin_label)
    
    # Standardize the test features (using the scaler from the fold)
    X_test = scaler.transform(X_test)
    
    # Perform inference
    test_dataset = GeneExpressionDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Accumulate absolute errors for each unique value of the target variable (age_bin_label)
    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            outputs = model(batch_features)
            preds = corn_label_from_logits(outputs)

            # Calculate absolute errors
            abs_errors = torch.abs(preds - batch_labels.float())

            # Update error sums and counts for each label in the batch
            for i, label in enumerate(batch_labels.cpu().numpy()):
                error_sums[label] += abs_errors[i].item()
                counts[label] += 1

# Calculate MAE for each unique value of the response variable (age_bin_label)
mae_per_label = {label: error_sums[label] / counts[label] if counts[label] > 0 else 0 for label in unique_age_bin_labels}

# Print MAE for each label
for label, mae in mae_per_label.items():
    print(f"MAE for label {label}: {mae:.4f}")

MAE for label 0: 1.0829
MAE for label 1: 0.7539
MAE for label 2: 0.6630
MAE for label 3: 0.5614
MAE for label 4: 0.7568
MAE for label 5: 0.6282
MAE for label 6: 0.4921
MAE for label 7: 0.7117
MAE for label 8: 1.0032
MAE for label 9: 1.0414


In [29]:
refine_labels2['age_bin'].value_counts()

age_bin
(45–60]    2711
(12–20]    2157
(20–35]    1867
(60–70]    1554
(35–45]    1404
[0–2]      1243
(8–12]     1098
(70–80]     929
(2–8]       760
> 80        483
Name: count, dtype: int64

In [30]:
counts

{0: 1243,
 1: 760,
 2: 1098,
 3: 2157,
 4: 1867,
 5: 1404,
 6: 2711,
 7: 1554,
 8: 929,
 9: 483}

## Normal MLP

In [185]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import mean_absolute_error

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Input layer
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(10, activation='softmax'))  # Output layer for 10 classes

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.1)

# Predict on test data
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Test MAE: {mae:.4f}')

2024-10-20 21:34:20.641664: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/davydsadovskyy/krishnanLab/age_sex_proteomic_model/venv_ordinal_neural_network/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.3691 - loss: 2.4832 - val_accuracy: 0.5198 - val_loss: 1.5153
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5789 - loss: 1.2933 - val_accuracy: 0.5638 - val_loss: 1.3785
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6023 - loss: 1.1207 - val_accuracy: 0.5831 - val_loss: 1.2493
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6432 - loss: 1.0054 - val_accuracy: 0.5884 - val_loss: 1.3479
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6638 - loss: 0.9608 - val_accuracy: 0.6139 - val_loss: 1.2630
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6929 - loss: 0.8587 - val_accuracy: 0.6218 - val_loss: 1.3022
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7003 - loss: 0.8371 - val_accuracy: 0.6060 - val_loss: 1.3343
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7223 - loss: 0.7526 - val_accuracy: 0.6165 - v

### refine (RNAseq)

#### MAE of arcsin with scaler:
#### MAE of arcsin without scaler:
#### MAE of raw with scaler: 0.8146
#### MAE of raw without scaler:

## How good is regular MLP across each age bin??

In [31]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize accumulators for MAE
overall_mae_sum = 0
fold_mae_per_label = {label: 0 for label in np.unique(labels)}
count_per_label = {label: 0 for label in np.unique(labels)}

# Perform 5-fold cross-validation
for fold, (train_idx, test_idx) in enumerate(skf.split(refine2, labels)):
    print(f"Fold {fold + 1}")

    # Create train and test sets
    X_train, X_test = refine2[train_idx], refine2[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Define the model
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='softmax'))  # Output layer for 10 age bins

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss=SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.1, verbose=0)

    # Predict on test data
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Calculate overall MAE for this fold
    fold_mae = mean_absolute_error(y_test, y_pred)
    overall_mae_sum += fold_mae
    print(f'Test MAE for fold {fold + 1}: {fold_mae:.4f}')

    # Calculate MAE per age group (age_bin_label) for this fold
    for label in np.unique(y_test):
        label_indices = np.where(y_test == label)
        label_mae = mean_absolute_error(y_test[label_indices], y_pred[label_indices])
        fold_mae_per_label[label] += label_mae * len(label_indices[0])
        count_per_label[label] += len(label_indices[0])

# Calculate overall MAE across all folds
overall_mae = overall_mae_sum / 5
print(f'Overall MAE across 5 folds: {overall_mae:.4f}')

# Calculate MAE per age group across all folds
mae_per_label = {label: fold_mae_per_label[label] / count_per_label[label] if count_per_label[label] > 0 else 0 for label in fold_mae_per_label}

# Print MAE per age group
for label, mae in mae_per_label.items():
    print(f'MAE for age_bin_label {label}: {mae:.4f}')


Fold 1


/Users/davydsadovskyy/krishnanLab/age_sex_proteomic_model/venv_ordinal_neural_network/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test MAE for fold 1: 0.8906
Fold 2


/Users/davydsadovskyy/krishnanLab/age_sex_proteomic_model/venv_ordinal_neural_network/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test MAE for fold 2: 0.9835
Fold 3


...

89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test MAE for fold 3: 0.8398
Fold 4


/Users/davydsadovskyy/krishnanLab/age_sex_proteomic_model/venv_ordinal_neural_network/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test MAE for fold 4: 0.8969
Fold 5


/Users/davydsadovskyy/krishnanLab/age_sex_proteomic_model/venv_ordinal_neural_network/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test MAE for fold 5: 0.8874
Overall MAE across 5 folds: 0.8996
MAE for age_bin_label 0: 1.2373
MAE for age_bin_label 1: 0.6895
MAE for age_bin_label 2: 0.6521
MAE for age_bin_label 3: 0.6078
MAE for age_bin_label 4: 0.8575
MAE for age_bin_label 5: 1.0370
MAE for age_bin_label 6: 0.8384
MAE for age_bin_label 7: 0.9588
MAE for age_bin_label 8: 1.2573
MAE for age_bin_label 9: 1.4555


## Compare and Plot the Overall and age-bin-wise MAE for CORN and MLP

In [50]:
import pandas as pd

# CORN results from previous output (manually extracted)
corn_mae_per_label = {
    0: 1.0829,
    1: 0.7539,
    2: 0.6630,
    3: 0.5614,
    4: 0.7568,
    5: 0.6282,
    6: 0.4921,
    7: 0.7117,
    8: 1.0032,
    9: 1.0414
}

# Calculate CORN MAE by averaging across each fold
fold_data = joblib.load('fold_data.pkl')
corn_overall_mae = sum(fold_data['mae']) / len(fold_data['mae'])

# Data for MLP is already stored in overall_mae and mae_per_label from the MLP code

# Get value counts for each age bin directly from refine_labels2['age_bin'].value_counts()
age_bin_counts = refine_labels2['age_bin'].value_counts()

# Sort the counts by the integer labels (from refine_labels2['age_bin_label'])
sorted_bins = refine_labels2.groupby('age_bin_label')['age_bin'].first().sort_index()
sorted_bin_counts = refine_labels2['age_bin_label'].value_counts().sort_index()

# Combine CORN and MLP results into a single table, sorted by the smallest to largest age bin
comparison_data = {
    'Age Bin Label': sorted_bins.values,  # Sorted age bin labels
    'Count': sorted_bin_counts.values,  # Sorted counts for each bin
    'CORN MAE': [corn_mae_per_label[i] for i in range(10)],  # Sorted CORN MAE
    'MLP MAE': [mae_per_label[i] for i in range(10)]  # Sorted MLP MAE
}

# Create the comparison DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Add overall MAE to the table
overall_comparison_df = pd.DataFrame({
    'Model': ['CORN', 'MLP'],
    'Overall MAE': [corn_overall_mae, overall_mae]
})

# Display the detailed per-label MAE comparison with counts, sorted by age bin
print("5 Fold CV - Ordinal Neural Network (CORN) vs MLP")
print(comparison_df)

# Display the overall MAE comparison
print("\nOverall MAE Comparison:")
print(overall_comparison_df)


5 Fold CV - Ordinal Neural Network (CORN) vs MLP
  Age Bin Label  Count  CORN MAE   MLP MAE
0         [0–2]   1243    1.0829  1.237329
1         (2–8]    760    0.7539  0.689474
2        (8–12]   1098    0.6630  0.652095
3       (12–20]   2157    0.5614  0.607789
4       (20–35]   1867    0.7568  0.857525
5       (35–45]   1404    0.6282  1.037037
6       (45–60]   2711    0.4921  0.838436
7       (60–70]   1554    0.7117  0.958816
8       (70–80]    929    1.0032  1.257266
9          > 80    483    1.0414  1.455487

Overall MAE Comparison:
  Model  Overall MAE
0  CORN     0.711673
1   MLP     0.899621


#### averaging the per age bin MAE is not the same as averaging the MAE across 5 folds, why?

In [57]:
print(sum([corn_mae_per_label[i] for i in range(10)])/10)
print(sum([mae_per_label[i] for i in range(10)])/10)

0.76946
0.9591252898733315


In [59]:
print(sum([corn_mae_per_label[i] for i in range(10)])/10 - 0.711673)
print(sum([mae_per_label[i] for i in range(10)])/10 - .899621)

0.05778700000000003
0.05950428987333145
